In [0]:
ingestion_config_path = "abfss://configs@storageottanalytics.dfs.core.windows.net/config/ingestion_config.json"

queries = []

# Read ingestion config
ingestion_config_df = (
    spark.read
    .option("multiline", "true")
    .json(ingestion_config_path)
)

ingestion_config = [row.asDict() for row in ingestion_config_df.collect()]

for row in ingestion_config:
    table_name = row["table_name"]
    path_name = row["source_path"]

    # READ STREAM (Auto Loader)
    df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .option(
            "cloudFiles.schemaLocation",
            f"abfss://bronze@storageottanalytics.dfs.core.windows.net/schema/{table_name}"
        )
        .option(
            "cloudFiles.schemaEvolutionMode",
            "addNewColumns"
        )
        
        .load(path_name)
    )

    # WRITE STREAM (Delta)
    query = (
        df.writeStream
        .format("delta")
        .option(
            "checkpointLocation",
            f"abfss://bronze@storageottanalytics.dfs.core.windows.net/checkpoints/streaming/{table_name}"
        )
        .trigger(processingTime="10 seconds")
        .start(
            f"abfss://bronze@storageottanalytics.dfs.core.windows.net/{table_name}"
        )
    )

    queries.append(query)

# Keep notebook alive & monitor streams
for q in queries:
    q.awaitTermination()
